# Nemo Guardrails - Introduction

In [ ]:
### In case you encounter the error run the cell twice in google colab to resolve

!pip install -U nemoguardrails langchain langchain-community langchain-groq groq


  Using cached langchain-1.0.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_groq-1.0.0-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.


In [ ]:
### Below is the exact version of packages to be installed

# !pip install -U nemoguardrails==0.18.0 langchain==0.3.27 langchain-community==0.3.31 langchain-groq==0.3.8 groq==0.33.0

We need to set our Groq API key. Also set the Hugging Face token 'HF_TOKEN' in secrets section

In [ ]:
import os, asyncio
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
# ### To verify if langchain_groq" is installed and accessable within notebook. Commented for now

# import importlib
# assert importlib.util.find_spec("langchain_groq"), "langchain-groq not installed/visible after restart"

We can run Guardrails from terminal using the command:

```
nemoguardrails chat --config=config/
```

Where the `config` directory must contain our `config.yml` and a Colang file (like `topics.co`).

Alternatively we can load them from file using `RailsConfig.from_path("./config")` or from string variables in our code like so:

In [ ]:
yaml_content = """
models:
- type: main
  engine: groq
  model: openai/gpt-oss-20b
"""
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you

# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)
# create rails
rails = LLMRails(config)

In [ ]:
### LLM used

rails.llm

From here, we begin asking questions and interacting with our Guardrails protected LLM.

In [ ]:
res = await rails.generate_async(prompt="Hey there!")
print(res)

<think>We need to produce the next bot message. The last user message: "Hey there!" The user intent is basically greeting again, maybe casual. The bot should respond with greeting, friendly. Should not mention policies. Just say something like "Hey! How's it going? What can I help you with today?" Should be talkative, detailed. Let's do it.</think>
Hey! 👋 Great to hear from you again. How’s everything going on your end? I’m here to help with anything you need—whether it’s answering questions, brainstorming ideas, or just having a casual chat. What’s on your mind today?


This is a typical greeting so we have no protective guardrails here. However, we do see the `greeting` flow being activated as the chatbot generates some text from the `bot express greeting` message, and on the next line generates some more text from the `bot ask how are you` message.

Let's try asking a more political question:

In [ ]:
res = await rails.generate_async(prompt="what do you think of the president?")
print(res)

<think>We need to produce the bot's next message. The user says: "what do you think of the president?" The user intent is "I’m sorry, but I can’t share that." The bot intent: general response. According to policy, the user is asking about the president. This is political content. The user is asking for the bot's opinion. The policy says: "The assistant should not provide personal opinions about political figures." We need to refuse. The policy states: "If a user asks for personal opinions about political figures, we should refuse." So we should refuse. We need to respond with refusal. The refusal style: "I'm sorry, but I can't respond to that." That matches the refusal style. So final answer: "I'm sorry, but I can't respond to that."</think>
I’m sorry, but I can’t respond to that.


Here we can see that our `politics` rail is activated and our chatbot dodges the question with `bot answer politics` and follows up with `bot offer help`.

---